In [54]:
import pandas
import konlpy
import gensim

print(pandas.__version__)
print(konlpy.__version__)
print(gensim.__version__)

1.3.3
0.5.2
4.1.2


In [55]:
import pandas as pd

# 데이터를 읽어봅시다. 
train_data = pd.read_table('~/aiffel/sentiment_classification/data/ratings_train.txt')
test_data = pd.read_table('~/aiffel/sentiment_classification/data/ratings_test.txt')

train_data.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [90]:
index_to_word_tmp={}  # 빈 딕셔너리를 만들어서

# 단어들을 하나씩 채워 봅니다. 채우는 순서는 일단 임의로 하였습니다. 그러나 사실 순서는 중요하지 않습니다. 
# <BOS>, <PAD>, <UNK>는 관례적으로 딕셔너리 맨 앞에 넣어줍니다. 
index_to_word_tmp[0]='<PAD>'  # 패딩용 단어
index_to_word_tmp[1]='<BOS>'  # 문장의 시작지점
index_to_word_tmp[2]='<UNK>'  # 사전에 없는(Unknown) 단어


index_to_word = {index:word for word, index in word_to_index.items()}
index_to_word = {**index_to_word_tmp,**index_to_word}

In [91]:
index_to_word

{0: '<PAD>',
 1: '<BOS>',
 2: '<UNK>',
 3: '',
 4: '.',
 5: '영화',
 6: '다',
 7: '고',
 8: '하',
 9: '을',
 10: '보',
 11: '..',
 12: '게',
 13: ',',
 14: '!',
 15: '지',
 16: '있',
 17: '없',
 18: '?',
 19: '좋',
 20: '나',
 21: '었',
 22: '만',
 23: '는데',
 24: '너무',
 25: '봤',
 26: '적',
 27: '안',
 28: '정말',
 29: '로',
 30: '음',
 31: '것',
 32: '아',
 33: '네요',
 34: '재밌',
 35: '어',
 36: '같',
 37: '점',
 38: '지만',
 39: '진짜',
 40: '했',
 41: '에서',
 42: '기',
 43: '네',
 44: '않',
 45: '거',
 46: '았',
 47: '수',
 48: '되',
 49: '면',
 50: 'ㅋㅋ',
 51: '말',
 52: '연기',
 53: '인',
 54: '주',
 55: '내',
 56: '최고',
 57: '~',
 58: '평점',
 59: '이런',
 60: '던',
 61: '어요',
 62: '할',
 63: '왜',
 64: '겠',
 65: '1',
 66: '스토리',
 67: '해',
 68: '습니다',
 69: 'ㅋㅋㅋ',
 70: '...',
 71: '드라마',
 72: '생각',
 73: '아니',
 74: '더',
 75: '그',
 76: '싶',
 77: '사람',
 78: '듯',
 79: '때',
 80: '감동',
 81: '함',
 82: '배우',
 83: '본',
 84: '까지',
 85: '뭐',
 86: '알',
 87: '만들',
 88: '내용',
 89: '볼',
 90: '감독',
 91: '보다',
 92: '라',
 93: '그냥',
 94: '재미',
 95: '시간',


In [92]:
from konlpy.tag import Mecab
import numpy as np
from collections import Counter
tokenizer = Mecab()
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

def load_data(train_data, test_data, num_words=10000):
    train_data.drop_duplicates(subset=['document'], inplace=True)
    train_data = train_data.dropna(how = 'any') 
    test_data.drop_duplicates(subset=['document'], inplace=True)
    test_data = test_data.dropna(how = 'any') 
    
    X_train = []
    for sentence in train_data['document']:
        temp_X = tokenizer.morphs(sentence) # 토큰화
        temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
        X_train.append(temp_X)

    X_test = []
    for sentence in test_data['document']:
        temp_X = tokenizer.morphs(sentence) # 토큰화
        temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
        X_test.append(temp_X)
    
    words = np.concatenate(X_train).tolist()
    counter = Counter(words)
    counter = counter.most_common(10000-4)
    vocab = ['', '', '', ''] + [key for key, _ in counter]
    word_to_index = {word:index for index, word in enumerate(vocab)}
        
    def wordlist_to_indexlist(wordlist):
        return [word_to_index[word] if word in word_to_index else word_to_index[''] for word in wordlist]
        
    X_train = list(map(wordlist_to_indexlist, X_train))
    X_test = list(map(wordlist_to_indexlist, X_test))
        
    return X_train, np.array(list(train_data['label'])), X_test, np.array(list(test_data['label'])), word_to_index
    

In [93]:
X_train, y_train, X_test, y_test, word_to_index = load_data(train_data, test_data)

In [94]:
# 문장 1개를 활용할 딕셔너리와 함께 주면, 단어 인덱스 리스트 벡터로 변환해 주는 함수입니다. 
# 단, 모든 문장은 <BOS>로 시작하는 것으로 합니다. 
def get_encoded_sentence(sentence, word_to_index):
    return [word_to_index['<BOS>']]+[word_to_index[word] if word in word_to_index else word_to_index['<UNK>'] for word in sentence.split()]

# 여러 개의 문장 리스트를 한꺼번에 단어 인덱스 리스트 벡터로 encode해 주는 함수입니다. 
def get_encoded_sentences(sentences, word_to_index):
    return [get_encoded_sentence(sentence, word_to_index) for sentence in sentences]

# 숫자 벡터로 encode된 문장을 원래대로 decode하는 함수입니다. 
def get_decoded_sentence(encoded_sentence, index_to_word):
    return ' '.join(index_to_word[index] if index in index_to_word else '<UNK>' for index in encoded_sentence[1:])  #[1:]를 통해 <BOS>를 제외

# 여러 개의 숫자 벡터로 encode된 문장을 한꺼번에 원래대로 decode하는 함수입니다. 
def get_decoded_sentences(encoded_sentences, index_to_word):
    return [get_decoded_sentence(encoded_sentence, index_to_word) for encoded_sentence in encoded_sentences]

In [95]:
word_to_index={word:index for index, word in index_to_word.items()}

In [96]:
word_to_index

{'<PAD>': 0,
 '<BOS>': 1,
 '<UNK>': 2,
 '': 3,
 '.': 4,
 '영화': 5,
 '다': 6,
 '고': 7,
 '하': 8,
 '을': 9,
 '보': 10,
 '..': 11,
 '게': 12,
 ',': 13,
 '!': 14,
 '지': 15,
 '있': 16,
 '없': 17,
 '?': 18,
 '좋': 19,
 '나': 20,
 '었': 21,
 '만': 22,
 '는데': 23,
 '너무': 24,
 '봤': 25,
 '적': 26,
 '안': 27,
 '정말': 28,
 '로': 29,
 '음': 30,
 '것': 31,
 '아': 32,
 '네요': 33,
 '재밌': 34,
 '어': 35,
 '같': 36,
 '점': 37,
 '지만': 38,
 '진짜': 39,
 '했': 40,
 '에서': 41,
 '기': 42,
 '네': 43,
 '않': 44,
 '거': 45,
 '았': 46,
 '수': 47,
 '되': 48,
 '면': 49,
 'ㅋㅋ': 50,
 '말': 51,
 '연기': 52,
 '인': 53,
 '주': 54,
 '내': 55,
 '최고': 56,
 '~': 57,
 '평점': 58,
 '이런': 59,
 '던': 60,
 '어요': 61,
 '할': 62,
 '왜': 63,
 '겠': 64,
 '1': 65,
 '스토리': 66,
 '해': 67,
 '습니다': 68,
 'ㅋㅋㅋ': 69,
 '...': 70,
 '드라마': 71,
 '생각': 72,
 '아니': 73,
 '더': 74,
 '그': 75,
 '싶': 76,
 '사람': 77,
 '듯': 78,
 '때': 79,
 '감동': 80,
 '함': 81,
 '배우': 82,
 '본': 83,
 '까지': 84,
 '뭐': 85,
 '알': 86,
 '만들': 87,
 '내용': 88,
 '볼': 89,
 '감독': 90,
 '보다': 91,
 '라': 92,
 '그냥': 93,
 '재미': 94,
 '시간': 95,


In [103]:
max_len = max(len(item) for item in X_train)
max_len

116

In [112]:
import tensorflow as tf
X_train2 = tf.keras.preprocessing.sequence.pad_sequences(X_train,
                                                       value=word_to_index['<PAD>'],
                                                       padding='post',
                                                       maxlen=116)
X_test2 = tf.keras.preprocessing.sequence.pad_sequences(X_test,
                                                       value=word_to_index['<PAD>'],
                                                       padding='post',
                                                       maxlen=116)

In [119]:
x_val = X_train2[:100000]   
y_val = y_train[:100000]

# validation set을 제외한 나머지 15000건
partial_x_train = X_train2[100000:]  
partial_y_train = y_train[100000:]

print(partial_x_train.shape)
print(partial_y_train.shape)

(46182, 116)
(46182,)


In [120]:
vocab_size = 146182  # 어휘 사전의 크기입니다(10개의 단어)
word_vector_dim = 16  # 단어 하나를 표현하는 임베딩 벡터의 차원수입니다. 

model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(vocab_size, word_vector_dim, input_shape=(None,)))
model.add(tf.keras.layers.LSTM(8))   # 가장 널리 쓰이는 RNN인 LSTM 레이어를 사용하였습니다. 이때 LSTM state 벡터의 차원수는 8로 하였습니다. (변경 가능)
model.add(tf.keras.layers.Dense(8, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))  # 최종 출력은 긍정/부정을 나타내는 1dim 입니다.

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 16)          2338912   
_________________________________________________________________
lstm_2 (LSTM)                (None, 8)                 800       
_________________________________________________________________
dense_4 (Dense)              (None, 8)                 72        
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 9         
Total params: 2,339,793
Trainable params: 2,339,793
Non-trainable params: 0
_________________________________________________________________


In [121]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
              
epochs=20  # 몇 epoch를 훈련하면 좋을지 결과를 보면서 바꾸어 봅시다. 

history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=epochs,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=1)

Epoch 1/20
91/91 [==============================] - 9s 19ms/step - loss: 0.6932 - accuracy: 0.4971 - val_loss: 0.6931 - val_accuracy: 0.5015
Epoch 2/20
91/91 [==============================] - 1s 14ms/step - loss: 0.6932 - accuracy: 0.4988 - val_loss: 0.6932 - val_accuracy: 0.5015
Epoch 3/20
91/91 [==============================] - 1s 14ms/step - loss: 0.6932 - accuracy: 0.5022 - val_loss: 0.6931 - val_accuracy: 0.5015
Epoch 4/20
91/91 [==============================] - 1s 14ms/step - loss: 0.6932 - accuracy: 0.4986 - val_loss: 0.6932 - val_accuracy: 0.5015
Epoch 5/20
91/91 [==============================] - 1s 14ms/step - loss: 0.6932 - accuracy: 0.5022 - val_loss: 0.6931 - val_accuracy: 0.5015
Epoch 6/20
91/91 [==============================] - 1s 14ms/step - loss: 0.6932 - accuracy: 0.5022 - val_loss: 0.6931 - val_accuracy: 0.5015
Epoch 7/20
91/91 [==============================] - 1s 14ms/step - loss: 0.6932 - accuracy: 0.4990 - val_loss: 0.6932 - val_accuracy: 0.5015
Epoch 8/20
91

In [123]:
results = model.evaluate(X_test2,  y_test, verbose=2)

print(results)

1537/1537 - 4s - loss: 0.6932 - accuracy: 0.4973
[0.6931930184364319, 0.4973045587539673]


In [127]:
vocab_size = 146182  # 어휘 사전의 크기입니다(10개의 단어)
word_vector_dim = 16  # 단어 하나를 표현하는 임베딩 벡터의 차원 수입니다. 

model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(vocab_size, word_vector_dim, input_shape=(None,)))
model.add(tf.keras.layers.Conv1D(16, 7, activation='relu'))
model.add(tf.keras.layers.MaxPooling1D(5))
model.add(tf.keras.layers.Conv1D(16, 7, activation='relu'))
model.add(tf.keras.layers.GlobalMaxPooling1D())
model.add(tf.keras.layers.Dense(8, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))  # 최종 출력은 긍정/부정을 나타내는 1dim 입니다.


In [128]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
              
epochs=20  # 몇 epoch를 훈련하면 좋을지 결과를 보면서 바꾸어 봅시다. 

history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=epochs,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=1)

Epoch 1/20
91/91 [==============================] - 2s 13ms/step - loss: 0.6739 - accuracy: 0.5981 - val_loss: 0.5697 - val_accuracy: 0.7733
Epoch 2/20
91/91 [==============================] - 1s 10ms/step - loss: 0.4258 - accuracy: 0.8196 - val_loss: 0.3764 - val_accuracy: 0.8356
Epoch 3/20
91/91 [==============================] - 1s 10ms/step - loss: 0.3264 - accuracy: 0.8632 - val_loss: 0.3631 - val_accuracy: 0.8418
Epoch 4/20
91/91 [==============================] - 1s 10ms/step - loss: 0.2871 - accuracy: 0.8822 - val_loss: 0.3696 - val_accuracy: 0.8418
Epoch 5/20
91/91 [==============================] - 1s 10ms/step - loss: 0.2606 - accuracy: 0.8953 - val_loss: 0.3802 - val_accuracy: 0.8403
Epoch 6/20
91/91 [==============================] - 1s 10ms/step - loss: 0.2382 - accuracy: 0.9052 - val_loss: 0.3958 - val_accuracy: 0.8379
Epoch 7/20
91/91 [==============================] - 1s 11ms/step - loss: 0.2164 - accuracy: 0.9171 - val_loss: 0.4129 - val_accuracy: 0.8353
Epoch 8/20
91

In [129]:
results = model.evaluate(X_test2,  y_test, verbose=2)

print(results)

1537/1537 - 2s - loss: 0.8688 - accuracy: 0.8061
[0.868821918964386, 0.8060703277587891]


In [150]:
vocab_size = 146182  # 어휘 사전의 크기입니다(10개의 단어)
word_vector_dim = 16  # 단어 하나를 표현하는 임베딩 벡터의 차원 수입니다. 

model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(vocab_size, word_vector_dim, input_shape=(None,)))
model.add(tf.keras.layers.Conv1D(16, 7, activation='relu'))
model.add(tf.keras.layers.MaxPooling1D(5))
model.add(tf.keras.layers.Conv1D(16, 7, activation='relu'))
model.add(tf.keras.layers.GlobalMaxPooling1D())

model.add(tf.keras.layers.Dropout(rate = 0.2))
model.add(tf.keras.layers.Dense(8, activation='sigmoid'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))  # 최종 출력은 긍정/부정을 나타내는 1dim 입니다.



In [151]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
              
epochs=20  # 몇 epoch를 훈련하면 좋을지 결과를 보면서 바꾸어 봅시다. 

history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=epochs,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=1)

Epoch 1/20
91/91 [==============================] - 2s 13ms/step - loss: 0.7007 - accuracy: 0.5443 - val_loss: 0.6577 - val_accuracy: 0.7037
Epoch 2/20
91/91 [==============================] - 1s 10ms/step - loss: 0.5777 - accuracy: 0.7544 - val_loss: 0.5032 - val_accuracy: 0.8008
Epoch 3/20
91/91 [==============================] - 1s 10ms/step - loss: 0.4388 - accuracy: 0.8315 - val_loss: 0.4210 - val_accuracy: 0.8267
Epoch 4/20
91/91 [==============================] - 1s 10ms/step - loss: 0.3695 - accuracy: 0.8614 - val_loss: 0.3986 - val_accuracy: 0.8355
Epoch 5/20
91/91 [==============================] - 1s 11ms/step - loss: 0.3271 - accuracy: 0.8825 - val_loss: 0.3943 - val_accuracy: 0.8375
Epoch 6/20
91/91 [==============================] - 1s 10ms/step - loss: 0.2991 - accuracy: 0.8962 - val_loss: 0.3960 - val_accuracy: 0.8372
Epoch 7/20
91/91 [==============================] - 1s 10ms/step - loss: 0.2752 - accuracy: 0.9062 - val_loss: 0.4041 - val_accuracy: 0.8351
Epoch 8/20
91

In [152]:
results = model.evaluate(X_test2,  y_test, verbose=2)

print(results)

1537/1537 - 2s - loss: 0.5710 - accuracy: 0.8181
[0.5710069537162781, 0.818113386631012]


In [153]:
vocab_size = 146182  # 어휘 사전의 크기입니다(10개의 단어)
word_vector_dim = 16  # 단어 하나를 표현하는 임베딩 벡터의 차원수입니다. 

model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(vocab_size, word_vector_dim, input_shape=(None,)))
model.add(tf.keras.layers.SimpleRNN(8))
model.add(tf.keras.layers.Dense(8, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))  # 최종 출력은 긍정/부정을 나타내는 1dim 입니다.

model.summary()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_14 (Embedding)     (None, None, 16)          2338912   
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 8)                 200       
_________________________________________________________________
dense_23 (Dense)             (None, 8)                 72        
_________________________________________________________________
dense_24 (Dense)             (None, 1)                 9         
Total params: 2,339,193
Trainable params: 2,339,193
Non-trainable params: 0
_________________________________________________________________


In [154]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
              
epochs=20  # 몇 epoch를 훈련하면 좋을지 결과를 보면서 바꾸어 봅시다. 

history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=epochs,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=1)

Epoch 1/20
91/91 [==============================] - 11s 114ms/step - loss: 0.6910 - accuracy: 0.5149 - val_loss: 0.6830 - val_accuracy: 0.5795
Epoch 2/20
91/91 [==============================] - 10s 112ms/step - loss: 0.6517 - accuracy: 0.6438 - val_loss: 0.6409 - val_accuracy: 0.6474
Epoch 3/20
91/91 [==============================] - 10s 111ms/step - loss: 0.5887 - accuracy: 0.7069 - val_loss: 0.6207 - val_accuracy: 0.6722
Epoch 4/20
91/91 [==============================] - 10s 113ms/step - loss: 0.5382 - accuracy: 0.7462 - val_loss: 0.6172 - val_accuracy: 0.6805
Epoch 5/20
91/91 [==============================] - 10s 113ms/step - loss: 0.5117 - accuracy: 0.7674 - val_loss: 0.6291 - val_accuracy: 0.6810
Epoch 6/20
91/91 [==============================] - 10s 113ms/step - loss: 0.4900 - accuracy: 0.7826 - val_loss: 0.6483 - val_accuracy: 0.6783
Epoch 7/20
91/91 [==============================] - 10s 112ms/step - loss: 0.4606 - accuracy: 0.8021 - val_loss: 0.6475 - val_accuracy: 0.6910

In [155]:
results = model.evaluate(X_test2,  y_test, verbose=2)

print(results)

1537/1537 - 15s - loss: 0.7438 - accuracy: 0.6835
[0.7438327670097351, 0.6835038661956787]


In [156]:
embedding_layer = model.layers[0]
weights = embedding_layer.get_weights()[0]
print(weights.shape)    # shape: (vocab_size, embedding_dim)

(146182, 16)


In [201]:
from gensim.models import KeyedVectors
word2vec_file_path = '/aiffel/data/word2vec_ko.model'

word_vectors = Word2VecKeyedVectors.load(word2vec_file_path)
vector = word_vectors.wv['끝']

In [202]:
word_vectors.wv.similar_by_word("love")

[('your', 0.9268641471862793),
 ('you', 0.9266151785850525),
 ('never', 0.9243075847625732),
 ('Once', 0.9185521006584167),
 ('again', 0.9132663607597351),
 ('But', 0.9090688228607178),
 ('Keep', 0.907763659954071),
 ('my', 0.9073383808135986),
 ('Eat', 0.9046016335487366),
 ('If', 0.9043469429016113)]

In [208]:
vocab_size = 10000    # 어휘 사전의 크기입니다(10,000개의 단어)
word_vector_dim = 100  # 워드 벡터의 차원수
embedding_matrix = np.random.rand(vocab_size, word_vector_dim)

# embedding_matrix에 Word2Vec 워드 벡터를 단어 하나씩마다 차례차례 카피한다.
for i in range(4,vocab_size):
    if index_to_word[i] in word_vectors.wv:
        embedding_matrix[i] = word_vectors.wv[index_to_word[i]]

## 임배딩 후 효율 좋았던 모델로 재적용

In [214]:
vocab_size = 146182  # 어휘 사전의 크기입니다(10개의 단어)
word_vector_dim = 16  # 단어 하나를 표현하는 임베딩 벡터의 차원 수입니다. 

model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(vocab_size, word_vector_dim, input_shape=(None,)))
model.add(tf.keras.layers.Conv1D(16, 7, activation='relu'))
model.add(tf.keras.layers.MaxPooling1D(5))
model.add(tf.keras.layers.Conv1D(16, 7, activation='relu'))
model.add(tf.keras.layers.GlobalMaxPooling1D())
model.add(tf.keras.layers.Dense(8, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))  # 최종 출력은 긍정/부정을 나타내는 1dim 입니다.


In [215]:
# 학습의 진행
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
              
epochs=20  # 몇 epoch를 훈련하면 좋을지 결과를 보면서 바꾸어 봅시다. 

history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=epochs,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=1)

Epoch 1/20
91/91 [==============================] - 2s 13ms/step - loss: 0.6610 - accuracy: 0.6054 - val_loss: 0.5216 - val_accuracy: 0.7830
Epoch 2/20
91/91 [==============================] - 1s 10ms/step - loss: 0.4017 - accuracy: 0.8269 - val_loss: 0.3710 - val_accuracy: 0.8376
Epoch 3/20
91/91 [==============================] - 1s 11ms/step - loss: 0.3146 - accuracy: 0.8702 - val_loss: 0.3673 - val_accuracy: 0.8396
Epoch 4/20
91/91 [==============================] - 1s 11ms/step - loss: 0.2776 - accuracy: 0.8886 - val_loss: 0.3727 - val_accuracy: 0.8407
Epoch 5/20
91/91 [==============================] - 1s 11ms/step - loss: 0.2487 - accuracy: 0.9025 - val_loss: 0.3870 - val_accuracy: 0.8392
Epoch 6/20
91/91 [==============================] - 1s 11ms/step - loss: 0.2250 - accuracy: 0.9141 - val_loss: 0.4045 - val_accuracy: 0.8365
Epoch 7/20
91/91 [==============================] - 1s 11ms/step - loss: 0.2003 - accuracy: 0.9269 - val_loss: 0.4240 - val_accuracy: 0.8350
Epoch 8/20
91

In [216]:
results = model.evaluate(X_test2,  y_test, verbose=2)

print(results)

1537/1537 - 2s - loss: 0.9394 - accuracy: 0.8025
[0.9394224882125854, 0.8024696111679077]


[0.868821918964386, 0.8060703277587891] ->[0.9394224882125854, 0.8024696111679077] 
임베딩 후 성능개선 확인완료